---
---

## MEDICAL CODING POC

---
---

---

Extract Data from Snowflake

---


In [25]:
import snowflake.connector
import pandas as pd

In [26]:
conn = snowflake.connector.connect(
user = 'DOLPHIN',
password = 'Maapaa@1603',  # Avoid hardcoding sensitive information
account = 'URB63596',
database = 'mimic_iv_medi_assist',
schema = 'staging_mimc',
warehouse = 'ANIMAL_TASK_WH'
)

In [27]:
# Query to extract clinical discharge notes from STG_DISCHARGE
query = """
SELECT SUBJECT_ID, HADM_ID, TEXT 
FROM MIMIC_IV_MEDI_ASSIST.STAGING_MIMIC.STG_DISCHARGE
WHERE TEXT IS NOT NULL
LIMIT 1
"""

In [28]:
# Fetch the data
df_discharge_notes = pd.read_sql(query, conn)

/var/folders/wj/v5dnvjg12ws3p1zszn_fvgt80000gn/T/ipykernel_20205/1287995978.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_discharge_notes = pd.read_sql(query, conn)


In [29]:
df_discharge_notes.head()

,SUBJECT_ID,HADM_ID,TEXT
0,15992303,22502053,\nName: ___ Unit No: ___\n \...


In [31]:
# Print the entire content of the 'TEXT' column for the first row
print(df_discharge_notes['TEXT'].iloc[0])


 
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: MEDICINE
 
Allergies: 
Amoxicillin / azithromycin
 
Attending: ___
 
Chief Complaint:
Liver Failure
 
Major Surgical or Invasive Procedure:
___: Bone marrow biopsy
___: Liver biopsy
___: EGD

 
History of Present Illness:
___ yo male with MVP and OSA who presented on ___ with
malaise and fevers to 104 after routine dental cleaning found to
have a transaminitis with LFT's >1000. Work up at the OSH was
negative including tick borne illness, and viral hepatitis. An
___ and RF were negative. They reported cyclical fevers despite
treatment with doxycycline. A ferritin was found to be 34,480.
HLH was considered by heme/onc and ID. A bone marrow biopsy was
performed and reportedly noted hemophagocytosis. Given worsening
liver failure, Decadron 20mg was started. He was started on
vitamin K and lactulose for his liver failure. A rash was noted

---

2. Preprocess the Data : vidhya

---

In [50]:
import scispacy
import spacy
#Core models
import en_core_sci_sm
import en_core_sci_md
#NER specific models
import en_ner_bc5cdr_md
#Tools for extracting & displaying data
from spacy import displacy
import pandas as pd

/Users/shreyajaiswal/Desktop/Start-to-Complete/MediAssist-Healthcare-Data-Solutions/.venv/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_sci_sm' (0.5.1) was trained with spaCy v3.4.1 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/Users/shreyajaiswal/Desktop/Start-to-Complete/MediAssist-Healthcare-Data-Solutions/.venv/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_sci_md' (0.5.1) was trained with spaCy v3.4.1 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy v

In [51]:
text = df_discharge_notes['TEXT'].iloc[0]
text

' \nName:  ___               Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   M\n \nService: MEDICINE\n \nAllergies: \nAmoxicillin / azithromycin\n \nAttending: ___\n \nChief Complaint:\nLiver Failure\n \nMajor Surgical or Invasive Procedure:\n___: Bone marrow biopsy\n___: Liver biopsy\n___: EGD\n\n \nHistory of Present Illness:\n___ yo male with MVP and OSA who presented on ___ with\nmalaise and fevers to 104 after routine dental cleaning found to\nhave a transaminitis with LFT\'s >1000. Work up at the OSH was\nnegative including tick borne illness, and viral hepatitis. An\n___ and RF were negative. They reported cyclical fevers despite\ntreatment with doxycycline. A ferritin was found to be 34,480.\nHLH was considered by heme/onc and ID. A bone marrow biopsy was\nperformed and reportedly noted hemophagocytosis. Given worsening\nliver failure, Decadron 20mg was started. He was started on\nvitamin K and lactulose for 

In [54]:
nlp_sm = en_core_sci_sm.load()
doc = nlp_sm(text)

In [55]:
#Display resulting entity extraction
displacy_image = displacy.render(doc, jupyter=True,style='ent')

In [56]:
#Now Load the specific model: en_core_sci_md and pass text through
nlp_md = en_core_sci_md.load()
doc = nlp_md(text)

In [57]:
#Display resulting entity extraction
displacy_image = displacy.render(doc, jupyter=True,style='ent')

In [58]:
#Now Load specific model: import en_ner_bc5cdr_md and pass text through

nlp_bc = en_ner_bc5cdr_md.load()
doc = nlp_bc(text)
#Display resulting entity extraction
displacy_image = displacy.render(doc, jupyter=True,style='ent')

In [59]:
#Display the entity

print("TEXT", "START", "END", "ENTITY TYPE")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

TEXT START END ENTITY TYPE
Allergies 165 174 DISEASE
Amoxicillin 177 188 CHEMICAL
azithromycin 191 203 CHEMICAL
MVP 393 396 DISEASE
OSA 401 404 DISEASE
fevers 443 449 DISEASE
transaminitis 503 516 DISEASE
hepatitis 607 616 DISEASE
fevers 670 676 DISEASE
doxycycline 700 711 CHEMICAL
HLH 748 751 DISEASE
hemophagocytosis 843 859 DISEASE
liver failure 877 890 DISEASE
vitamin K 937 946 CHEMICAL
lactulose 951 960 CHEMICAL
liver failure 969 982 DISEASE
rash 986 990 DISEASE
epistaxis 1076 1085 DISEASE
silver 1106 1112 CHEMICAL
nitrate 1113 1120 CHEMICAL
hematemesis 1168 1179 DISEASE
hematochezia 1184 1196 DISEASE
etoposide 1278 1287 CHEMICAL
Lactulose 1488 1497 CHEMICAL
hematochezia 1537 1549 DISEASE
transaminitis 1601 1614 DISEASE
bilirubin 1651 1660 CHEMICAL
mitral valve prolapse 1787 1808 DISEASE
sleep apnea 1809 1820 DISEASE
-Colon cancer 1882 1895 DISEASE
icteric 2013 2020 DISEASE
ABDM 2178 2182 CHEMICAL
ascites 2212 2219 DISEASE
swelling 2234 2242 DISEASE
erythema 2246 2254 DISEASE
EXTR 

---

2. Preprocess the Data

---

In [32]:
import spacy
import re
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

In [33]:
# Ensure necessary NLTK packages are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shreyajaiswal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shreyajaiswal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shreyajaiswal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [37]:
text = df_discharge_notes['TEXT'].iloc[0]
text

' \nName:  ___               Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   M\n \nService: MEDICINE\n \nAllergies: \nAmoxicillin / azithromycin\n \nAttending: ___\n \nChief Complaint:\nLiver Failure\n \nMajor Surgical or Invasive Procedure:\n___: Bone marrow biopsy\n___: Liver biopsy\n___: EGD\n\n \nHistory of Present Illness:\n___ yo male with MVP and OSA who presented on ___ with\nmalaise and fevers to 104 after routine dental cleaning found to\nhave a transaminitis with LFT\'s >1000. Work up at the OSH was\nnegative including tick borne illness, and viral hepatitis. An\n___ and RF were negative. They reported cyclical fevers despite\ntreatment with doxycycline. A ferritin was found to be 34,480.\nHLH was considered by heme/onc and ID. A bone marrow biopsy was\nperformed and reportedly noted hemophagocytosis. Given worsening\nliver failure, Decadron 20mg was started. He was started on\nvitamin K and lactulose for 

In [36]:
# Convert text to lowercase
text = text.lower()
text

' \nname:  ___               unit no:   ___\n \nadmission date:  ___              discharge date:   ___\n \ndate of birth:  ___             sex:   m\n \nservice: medicine\n \nallergies: \namoxicillin / azithromycin\n \nattending: ___\n \nchief complaint:\nliver failure\n \nmajor surgical or invasive procedure:\n___: bone marrow biopsy\n___: liver biopsy\n___: egd\n\n \nhistory of present illness:\n___ yo male with mvp and osa who presented on ___ with\nmalaise and fevers to 104 after routine dental cleaning found to\nhave a transaminitis with lft\'s >1000. work up at the osh was\nnegative including tick borne illness, and viral hepatitis. an\n___ and rf were negative. they reported cyclical fevers despite\ntreatment with doxycycline. a ferritin was found to be 34,480.\nhlh was considered by heme/onc and id. a bone marrow biopsy was\nperformed and reportedly noted hemophagocytosis. given worsening\nliver failure, decadron 20mg was started. he was started on\nvitamin k and lactulose for 

In [38]:
# Remove special characters and numbers
text = re.sub(r'[^a-z\s]', '', text)
text

' \name                 nit o   \n \ndmission ate                ischarge ate   \n \nate of irth               ex   \n \nervice \n \nllergies \nmoxicillin  azithromycin\n \nttending \n \nhief omplaint\niver ailure\n \najor urgical or nvasive rocedure\n one marrow biopsy\n iver biopsy\n \n\n \nistory of resent llness\n yo male with  and  who presented on  with\nmalaise and fevers to  after routine dental cleaning found to\nhave a transaminitis with s  ork up at the  was\nnegative including tick borne illness and viral hepatitis n\n and  were negative hey reported cyclical fevers despite\ntreatment with doxycycline  ferritin was found to be \n was considered by hemeonc and   bone marrow biopsy was\nperformed and reportedly noted hemophagocytosis iven worsening\nliver failure ecadron mg was started e was started on\nvitamin  and lactulose for his liver failure  rash was noted\nbut not described on  n  his  rose to \nand he started to have epistaxis his resolved with silver\nnitrate sticks

In [43]:
# Remove extra whitespace
text = re.sub(r'\s+', ' ', text).strip()
print(text)

ame nit o dmission ate ischarge ate ate of irth ex ervice llergies moxicillin azithromycin ttending hief omplaint iver ailure ajor urgical or nvasive rocedure one marrow biopsy iver biopsy istory of resent llness yo male with and who presented on with malaise and fevers to after routine dental cleaning found to have a transaminitis with s ork up at the was negative including tick borne illness and viral hepatitis n and were negative hey reported cyclical fevers despite treatment with doxycycline ferritin was found to be was considered by hemeonc and bone marrow biopsy was performed and reportedly noted hemophagocytosis iven worsening liver failure ecadron mg was started e was started on vitamin and lactulose for his liver failure rash was noted but not described on n his rose to and he started to have epistaxis his resolved with silver nitrate sticks but then returned e had an episode of hematemesis and hematochezia e was given sent to the and was consulted e was then started on etopos

---

spacy

---

In [19]:
# Load a spaCy model for NLP processing
nlp = spacy.load("en_core_web_sm")

In [20]:
text = df_discharge_notes['TEXT'].iloc[0]

In [10]:
print(text)

 
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: MEDICINE
 
Allergies: 
Amoxicillin / azithromycin
 
Attending: ___
 
Chief Complaint:
Liver Failure
 
Major Surgical or Invasive Procedure:
___: Bone marrow biopsy
___: Liver biopsy
___: EGD

 
History of Present Illness:
___ yo male with MVP and OSA who presented on ___ with
malaise and fevers to 104 after routine dental cleaning found to
have a transaminitis with LFT's >1000. Work up at the OSH was
negative including tick borne illness, and viral hepatitis. An
___ and RF were negative. They reported cyclical fevers despite
treatment with doxycycline. A ferritin was found to be 34,480.
HLH was considered by heme/onc and ID. A bone marrow biopsy was
performed and reportedly noted hemophagocytosis. Given worsening
liver failure, Decadron 20mg was started. He was started on
vitamin K and lactulose for his liver failure. A rash was noted

In [21]:
doc = nlp(text)
print(doc)

 
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: MEDICINE
 
Allergies: 
Amoxicillin / azithromycin
 
Attending: ___
 
Chief Complaint:
Liver Failure
 
Major Surgical or Invasive Procedure:
___: Bone marrow biopsy
___: Liver biopsy
___: EGD

 
History of Present Illness:
___ yo male with MVP and OSA who presented on ___ with
malaise and fevers to 104 after routine dental cleaning found to
have a transaminitis with LFT's >1000. Work up at the OSH was
negative including tick borne illness, and viral hepatitis. An
___ and RF were negative. They reported cyclical fevers despite
treatment with doxycycline. A ferritin was found to be 34,480.
HLH was considered by heme/onc and ID. A bone marrow biopsy was
performed and reportedly noted hemophagocytosis. Given worsening
liver failure, Decadron 20mg was started. He was started on
vitamin K and lactulose for his liver failure. A rash was noted